#### Summary
- We will be getting data from the Openstreetmap API with the use of the "overpass-api".
- Create a new postgresql database to hold our data.
- Run sample SQL query to show all the restuarants present in "Cambridge" from our database.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import psycopg2 as psql #postgresql library
import requests # to get data from api
import pandas as pd

ModuleNotFoundError: No module named 'psycopg2'

### Create Database

In [ ]:
conn = None
cursor = None
try:
    conn = psql.connect(host="localhost",
                        user="postgres",
                        password="mynewpassword")
    conn.autocommit = True
    sql = '''CREATE database restaurant_database''';

    cursor = conn.cursor()
    #Creating a database
    cursor.execute(sql)
    print("Database created successfully........")

    #Closing the connection
    conn.close()
except Exception as error:
    print(error)
    
finally:
    if conn is not None:
        conn.close()
    if cursor is not None:
        cursor.close()

### Load data into database

In [3]:
# Access the api
overpass_url = 'http://overpass-api.de/api/interpreter'
overpass_query = """
    [out:json];
    area["ISO3166-1"="GB"][admin_level=2];
    node[amenity="restaurant"](area);
    out;"""
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# properly structure the data
data = data['elements']
for i,datum in enumerate(data):
    for key in ['lat', 'lon', 'tags']:
        if key not in datum.keys():
            del data[i]
            continue
    for key in ['name', 'addr:city']:
        if key not in datum['tags'].keys():
            data[i]['tags'][key] = ''

In [2]:
conn = None
cursor = None
try:
    conn = psql.connect(host="localhost",
                        user="postgres",
                        database = 'restaurant_database',
                        password="mynewpassword")
    conn.autocommit = True
    cursor = conn.cursor()
    
#     create a new table
    create_script = """CREATE TABLE IF NOT EXISTS restaurants_great_britain (
                        id int PRIMARY KEY,
                        lat FLOAT(23),
                        lon FLOAT(23),
                        name varchar(100),
                        city varchar(100))"""
    
    cursor.execute(create_script)
    
#     load data into the table
    new_data = [(row, datum['lat'], datum['lon'], datum['tags']['name'], datum['tags']['addr:city']) for row,datum in enumerate(data)]
    insert_script = "INSERT INTO restaurants_great_britain (id, lat, lon, name, city) VALUES (%s, %s, %s, %s, %s)"  
    for insert_value in new_data:
        cursor.execute(insert_script, insert_value)
    
except Exception as error:
    print(error)
    
finally:
    if conn is not None:
        conn.close()
    if cursor is not None:
        cursor.close()

name 'psql' is not defined


### Run Simple SQL QUERIES 

In [5]:
# query all restaurants present in Cambridge
conn = None
cursor = None
try:
    conn = psql.connect(host="localhost",
                        user="postgres",
                        database = 'restaurant_database',
                        password="mynewpassword")
    
    conn.autocommit = True
    cursor = conn.cursor()

    create_script = """SELECT * from restaurants_great_britain
                        WHERE city='Cambridge';"""
    cursor.execute(create_script)
    
    fetch = cursor.fetchall()
    df = pd.DataFrame(fetch, columns = ['id', 'lat', 'long', 'name', 'city'])
except Exception as error:
    print(error)
    
finally:
    if conn is not None:
        conn.close()
    if cursor is not None:
        cursor.close()

In [3]:
df

NameError: name 'df' is not defined

### Show queried restaurants

In [8]:
import folium
m = folium.Map(location=df.iloc[1][['lat','long']].astype('float').values.tolist(), zoom_start=13)

tooltip = "Click Here For More Info"

for row in range(len(df)):
    marker = folium.Marker(
        location=df.iloc[row][['lat','long']].astype('float').values.tolist(),
        popup="<stong>{}</stong>".format(df.iloc[row]['name']),
        tooltip=tooltip)
    marker.add_to(m)

m
